### Welcome to Week 5 Day 1

AutoGen AgentChat!

This should look simple and familiar, because it has a lot in common with Crew and OpenAI Agents SDK

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [2]:
# NB: This way we call the LLM from OpenAI
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [3]:
# NB: This way we create a model from Ollama
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [4]:
# This way we create the user message, note this is a text message
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 17, 10, 5, 2, 796428, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [5]:
# This way we create the Agent by importing the AssistantAgent class
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client, # Here we pass the LLM model created before
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True # Here we say that the result of the agent must be returned in the stream
)

### Put it all together with on_messages

In [ ]:
# Here we put all together to receive a response from the Agent
# CancellationToken lets you abort or interrupt long-running agent operations in AutoGen, 
# allowing graceful cancellation of processing (for example, if the user wants to stop a query).
from autogen_core import CancellationToken

# here we create the response that is of course an asyncronous process
# to do this we use the method .on_messasge on the agent
# we pass the message created before and the cancellation_token to interrupt the chat
response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content # here we receive the content of the reply

'Great choice! London is like a big, bustling, historical theme park—only with more rain and fewer roller coasters!'

### Let's make a local database of ticket prices
Here we create a sqlite DB to retrieve ticket prices.
It's a small example of RAG

In [7]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")  # Connect to (or create) the database file 'tickets.db'
c = conn.cursor()  # Create a cursor object to execute SQL commands. Now we can execute SQL commands on the database just created.
# Create the 'cities' table with city_name as the primary key and round_trip_price as a real number
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")  
conn.commit()  # Save (commit) the changes made to the database
conn.close()  # Close the connection to the database

In [8]:
# Populate our database
# We create a simple function to save records in the DB by passing city name and ticket price as parameters
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    # Insert or update the ticket price for a city in the database; replaces the record if the city already exists
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [9]:
# Method to get price for a city
# The method connects with the database, select the city based on its name and return it.
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [10]:
get_city_price("Rome")

499.0

In [12]:
# Now we're going to recreate the agent and equip it with tools
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True, # We want the resukt is returned back in the stream
    tools=[get_city_price], # This is the tool just created to retrieve the ticket price in the DB
    reflect_on_tool_use=True # Enables the agent to reflect on its previous tool use and adjust its responses accordingly
)

In [13]:
# Now we create the response with the asyncronous process
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
# These two lines allow to print the inner messages to understant what is going on
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_daIL366mfv0ijaEdKsljQAT4', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_daIL366mfv0ijaEdKsljQAT4', is_error=False)]


"A London trip? That's as cheap as a cuppa tea at £299! Just don't forget your umbrella! ☔️✈️"